In [2]:
# import dependencies

import pandas as pd
from config import api_key
import requests
import os
import math
from datetime import datetime
from config import finn
import finnhub
finnhub_client = finnhub.Client(api_key=finn)
import time
import datetime
now = datetime.datetime.now()
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import date

In [3]:
# url for api key
option_url = 'https://api.tdameritrade.com/v1/marketdata/chains'

In [4]:
# Set t equal to today

t = date.today()

# Read in volatility csv

vol = pd.read_csv('Sigma.csv')

In [5]:

# input conditions for options
symbol = input("Ticker")
strikecount = input("Strikecount")
expMonth = input('Exp Month')
contract = input('Contract type PUT, CALL, ALL')
strike = input("Strike Price")

Ticker
Strikecount12
Exp MonthALL
Contract type PUT, CALL, ALLALL
Strike Price


In [8]:
# Using last days price

sigma  = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["0"].tolist()[0]

arb = {}

# Set the start and end date
start_date = '2023-01-01'

t = date.today()

t = str(t)

# Get the data
data = yf.download(symbol, start_date, t)

close = data.tail(1)["Close"].values[0]


#uses api key to extract data
params = {'apikey':api_key,
         'symbol':symbol,
         'contractType': contract,
          'strikeCount': strikecount,
          'includeQuotes': False,
          'stratgey': 'SINGLE',
          'expMonth': expMonth,
          'optionType': 'S'


         }

content = requests.get(url = option_url, params=params)

data = content.json()




dates_ref = []

int_year = now.year
int_month = now.month
int_day = now.day

for i in data['putExpDateMap']:
    dates_ref.append(i)

for i in dates_ref:

    #create outline for 
    df = {}
    date_now = i[:10]
    volume = []
    theta = []
    gamma = []
    vega = []
    delta = []
    interest = []

    #iterate through each strike
    for strike in data["putExpDateMap"][i]:
        df[strike] = {'Put': '', 'Call':''}
        df[strike]['Put'] = data["putExpDateMap"][i][strike][0]["closePrice"]
        df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

    for strike in data["callExpDateMap"][i]:

        df[strike]['Call'] = data["callExpDateMap"][i][strike][0]["closePrice"]
        df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



    label = i

    strike_date = i.split('-')

    year = int(strike_date[0])

    month = int(strike_date[1])

    day = int(strike_date[2].split(':')[0])



    count = 0

    days = 0

    if year > int_year:
        count = year - int_year

        if month > int_month + 1:
            interval = month - 1

            for i in range(interval + 1):

                if i <= int_month:
                    continue
                else:
                    days += months_ref[i]

            diff_int_day = months_ref[int_month] - int_day

            days = days + day + diff_int_day

        else:
            for i in range(13):
                if i <= int_month:
                    continue
                else: 
                    days += months_ref[i]

            diff_int_day = months_ref[int_month] - int_day

            days += diff_int_day

            for i in range(1, month):
                days += months_ref[i]


            days += day

            days = days / 365

            count += days

    else:
        interval = month - 1

        for i in range(interval + 1):

            if i <= int_month:
                continue
            else:
                days += months_ref[i]

        diff_int_day = months_ref[int_month] - int_day

        days = days + day + diff_int_day

        days = days / 365

        count += days

    #convert to dataframe
    option_df = pd.DataFrame.from_dict(df, orient='index')

    for row, i in option_df.iterrows():


        K = float(row)

        C = i['Call']

        P = i['Put']
        
        t = 'p'

        true_put = blackScholes(r, close, K, count, sigma, t)
        
        t = 'c'

        true_call = blackScholes(r, close, K, count, sigma, t)
        
        diff_put = P - true_put
        
        diff_call = C - true_call
        
        arb[f'{label} - {K}'] = {'Put': df[row]['Put'], 'True Put' : true_put, 'Call': df[row]['Call'], 'True Call': true_call, 'Symbol': symbol, 'Call - Volume': df[row]['Call - Volume'], 'Put - Volume': df[row]['Put - Volume'], 'Price': close, "Difference Puts": diff_put, "Difference Calls": diff_call}



IndexError: list index out of range

In [6]:
tickers = pd.read_csv("S&P copy.csv")["S&P"].tolist()

In [7]:
optionable = pd.read_csv("Stocks.csv")["Tickers"].tolist()

In [8]:
common_list = set(tickers).intersection(optionable)

In [19]:


arb = {}

for symbol in common_list:
    
    sigma  = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["0"].tolist()[0]

    # Set the start and end date
    start_date = '2023-01-01'

    t = date.today()

    t = str(t)

    # Get the data
    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    while info["s"] == 'no_data':
        time.sleep(1)

        print(True)

        info = finnhub_client.stock_candles(symbol, 'D', test, test)

    data  = pd.DataFrame(info, index= [0])
    data.columns = ["Close", "High", "Low", "Open", "S", "Timestamp", "Volume"]
    data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
    data = data.drop(columns=["S", "Timestamp"])
    data = data.set_index("Datetime")

    close = data['Close'].values[0]


    #uses api key to extract data
    params = {'apikey':api_key,
             'symbol':symbol,
             'contractType': contract,
              'strikeCount': strikecount,
              'includeQuotes': False,
              'stratgey': 'SINGLE',
              'expMonth': expMonth,
              'optionType': 'S'


             }

    content = requests.get(url = option_url, params=params)

    data = content.json()




    dates_ref = []

    int_year = now.year
    int_month = now.month
    int_day = now.day

    for i in data['putExpDateMap']:
        dates_ref.append(i)

    for i in dates_ref:

        #create outline for 
        df = {}
        date_now = i[:10]
        volume = []
        theta = []
        gamma = []
        vega = []
        delta = []
        interest = []

        #iterate through each strike
        for strike in data["putExpDateMap"][i]:
            df[strike] = {'Put': '', 'Call':''}
            df[strike]['Put'] = data["putExpDateMap"][i][strike][0]["last"]
            df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

        for strike in data["callExpDateMap"][i]:

            df[strike]['Call'] = data["callExpDateMap"][i][strike][0]["last"]
            df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



        label = i

        strike_date = i.split('-')

        year = int(strike_date[0])

        month = int(strike_date[1])

        day = int(strike_date[2].split(':')[0])



        count = 0

        days = 0
        
        # obtain difference in time

        if year > int_year:
            count = year - int_year

            if month > int_month + 1:
                interval = month - 1

                for i in range(interval + 1):

                    if i <= int_month:
                        continue
                    else:
                        days += months_ref[i]

                diff_int_day = months_ref[int_month] - int_day

                days = days + day + diff_int_day

            else:
                for i in range(13):
                    if i <= int_month:
                        continue
                    else: 
                        days += months_ref[i]

                diff_int_day = months_ref[int_month] - int_day

                days += diff_int_day

                for i in range(1, month):
                    days += months_ref[i]


                days += day

                days = days / 365

                count += days

        else:
            interval = month - 1

            for i in range(interval + 1):

                if i <= int_month:
                    continue
                else:
                    days += months_ref[i]

            diff_int_day = months_ref[int_month] - int_day

            days = days + day + diff_int_day

            days = days / 365

            count += days

        #convert to dataframe
        option_df = pd.DataFrame.from_dict(df, orient='index')

        for row, i in option_df.iterrows():


            K = float(row)

            C = i['Call']

            P = i['Put']

            t = 'p'
            
            # obtain true put value using black scholes formula

            true_put = blackScholes(r, close, K, count, sigma, t)

            t = 'c'
            
            # obtain true call value using black scholes formula

            true_call = blackScholes(r, close, K, count, sigma, t)

            diff_put = P - true_put

            diff_call = C - true_call

            arb[f'{label} - {K}'] = {'Put': df[row]['Put'], 'True Put' : true_put, 'Call': df[row]['Call'], 'True Call': true_call, 'Symbol': symbol, 'Call - Volume': df[row]['Call - Volume'], 'Put - Volume': df[row]['Put - Volume'], 'Price': close, "Difference Puts": diff_put, "Difference Calls": diff_call}



/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_64181/49523710.py:30: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
/var/folders/g3/164

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


FinnhubAPIException: FinnhubAPIException(status_code: 503): JSON error message from Finnhub: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Rate Limited</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


<script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feed

In [ ]:
now = datetime.datetime.now()
int_year = now.year
int_month = now.month
int_day = now.day

In [62]:
strike_date

['2024', '03', '15:380']

In [55]:
now

datetime.datetime(2023, 2, 28, 8, 56, 15, 724781)

In [50]:
days

1.0410958904109588

In [ ]:
year = int(strike_date[0])

month = int(strike_date[1])

day = int(strike_date[2].split(':')[0])


In [47]:
year

2024

In [48]:
month

3

In [49]:
day

15

In [46]:
strike_date

['2024', '03', '15:380']

In [45]:
label

'2024-03-15:380'

In [44]:
row

'210.0'

In [42]:
count

2.0410958904109586

In [37]:
options_df = pd.DataFrame.from_dict(arb, orient='index')
options_df.head()

,Put,True Put,Call,True Call,Symbol,Call - Volume,Put - Volume,Price,Difference Puts,Difference Calls
2023-03-01:0 - 388.0,0.01,0.073722,6.24,6.623668,SPY,318,28590,394.5002,-0.063722,-0.383668
2023-03-01:0 - 389.0,0.01,0.134748,5.40,5.684822,SPY,543,25904,394.5002,-0.124748,-0.284822
2023-03-01:0 - 390.0,0.01,0.233579,4.71,4.783782,SPY,3052,78604,394.5002,-0.223579,-0.073782
2023-03-01:0 - 391.0,0.01,0.384869,3.63,3.935200,SPY,6545,62114,394.5002,-0.374869,-0.305200
2023-03-01:0 - 392.0,0.02,0.604260,2.60,3.154719,SPY,6669,181202,394.5002,-0.584260,-0.554719


In [38]:
options_df.sort_values(by="Difference Calls", ascending=False)

,Put,True Put,Call,True Call,Symbol,Call - Volume,Put - Volume,Price,Difference Puts,Difference Calls
2023-03-31:30 - 280.0,3.70,7.197297,51.51,41.548031,NFLX,0,1833,313.2400,-3.497297,9.961969
2023-03-24:23 - 280.0,2.92,5.446787,46.47,39.547096,NFLX,0,73,313.2400,-2.526787,6.922904
2023-03-31:30 - 290.0,5.45,10.267345,40.13,34.657748,NFLX,0,20,313.2400,-4.817345,5.472252
2023-03-31:30 - 300.0,8.63,14.087655,33.89,28.517726,NFLX,0,149,313.2400,-5.457655,5.372274
2023-03-24:23 - 290.0,4.52,8.232855,37.66,32.363889,NFLX,0,31,313.2400,-3.712855,5.296111
...,...,...,...,...,...,...,...,...,...,...
2024-03-15:380 - 320.0,37.10,45.767146,21.59,45.720044,QQQ,2,0,290.8000,-8.667146,-24.130044
2024-03-15:380 - 430.0,42.25,40.541832,20.08,44.216238,SPY,2,1,394.5002,1.708168,-24.136238
2024-03-15:380 - 325.0,41.27,48.432232,19.67,43.840644,QQQ,0,0,290.8000,-7.162232,-24.170644
2024-03-15:380 - 310.0,32.46,40.648532,25.50,49.690401,QQQ,3,1,290.8000,-8.188532,-24.190401


In [40]:
options_df.sort_values(by="Difference Calls", ascending=False).to_csv('Options.csv')

In [41]:
options_df.sort_values(by="Call - Volume", ascending=False).to_csv('Options.csv')

In [11]:
r = 0.048

In [12]:
def blackScholes(r, S, K, T, sigma, t, p):
    d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if t == "c":
        price = S*np.exp(-p*T)*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif t == "p":
        price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*np.exp(-p*T)*norm.cdf(-d1, 0, 1)
    return price


In [13]:
# Last days price

arb = {}

for symbol in common_list:
    sigma  = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["Sigma"].tolist()[0]
    
    mu  = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["Mu"].tolist()[0]
    
    div = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["Div"].tolist()[0]
    
    

    # Set the start and end date
    start_date = '2023-01-01'

    t = date.today()

    t = str(t)

    # Get the data
    data = yf.download(symbol, start_date, t)

    try:
        close = data.tail(1)["Close"].values[0]
    except:
        continue


    #uses api key to extract data
    params = {'apikey':api_key,
             'symbol':symbol,
             'contractType': contract,
              'strikeCount': strikecount,
              'includeQuotes': False,
              'stratgey': 'SINGLE',
              'expMonth': expMonth,
              'optionType': 'S'


             }

    content = requests.get(url = option_url, params=params)

    data = content.json()




    dates_ref = []

    int_year = now.year
    int_month = now.month
    int_day = now.day

    for i in data['putExpDateMap']:
        dates_ref.append(i)

    for i in dates_ref:

        #create outline for 
        df = {}
        date_now = i[:10]
        volume = []
        theta = []
        gamma = []
        vega = []
        delta = []
        interest = []

        #iterate through each strike
        for strike in data["putExpDateMap"][i]:
            df[strike] = {'Put': 0, "Call": 0, 'Call - Volume' : 0, 'Put - Volume' : 0}
            ask = data["putExpDateMap"][i][strike][0]["ask"]
            
            bid = data["putExpDateMap"][i][strike][0]["bid"]
            
            
            df[strike]['Put'] = (ask + bid) / 2
            df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

        for strike in data["callExpDateMap"][i]:
            
            if strike not in df:
                df[strike] = {'Put': 0, "Call": 0, 'Call - Volume' : 0, 'Put - Volume' : 0}

            ask = data["callExpDateMap"][i][strike][0]["ask"]
            
            bid = data["callExpDateMap"][i][strike][0]["bid"]

            df[strike]['Call'] = (ask + bid) / 2
            df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



        label = i

        strike_date = i.split('-')

        year = int(strike_date[0])

        month = int(strike_date[1])

        day = int(strike_date[2].split(':')[0])
        
        count = int(strike_date[2].split(':')[1]) / 365

        #convert to dataframe
        option_df = pd.DataFrame.from_dict(df, orient='index')

        for row, i in option_df.iterrows():


            K = float(row)

            C = i['Call']

            P = i['Put']

            t = 'p'
            
            # obtain true put value using black scholes formula

            true_put = blackScholes(r, close, K, count, sigma, t, div)

            t = 'c'
            
            # obtain true call value using black scholes formula

            true_call = blackScholes(r, close, K, count, sigma, t, div)

            diff_put = P - true_put

            diff_call = C - true_call
            
            mu1 = np.log(close) + (mu - 0.5 * sigma **2) * count

            d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))

            d_down = (np.log(K) - mu1) / (sigma)

            up = norm.cdf(d_up, 0, 1) * 100

            down = norm.cdf(d_down, 0, 1) * 100
            
            diff_put = (P - true_put) / P * 100

            diff_call = (C - true_call) / C * 100

            arb[f'{label} - {K}'] = {'Date': label, 'Strike': K, 'Put': df[row]['Put'], 'True Put' : true_put, 'Call': df[row]['Call'], 'True Call': true_call, 'Symbol': symbol, 'Call - Volume': df[row]['Call - Volume'], 'Put - Volume': df[row]['Put - Volume'], 'Price': close, "Difference Puts": diff_put, "Difference Calls": diff_call, "% be Below": down, "% be above": up}
            
    time.sleep(0.5)


[*********************100%***********************]  1 of 1 completed


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1592665972.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1484490205.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))


[*********************100%***********************]  1 of 1 completed


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1592665972.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1484490205.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))


[*********************100%***********************]  1 of 1 completed


...

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1592665972.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1484490205.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))


[*********************100%***********************]  1 of 1 completed


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1592665972.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1484490205.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))


[*********************100%***********************]  1 of 1 completed


/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1592665972.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_31243/1484490205.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  d_up = (np.log(close/K) + (mu - 0.5 * sigma **2) * (count)) / (sigma * math.sqrt(count))


In [33]:
# Current Price

arb = {}

for symbol in common_list:
    
    sigma  = vol.loc[vol["Unnamed: 0"] == f"{symbol}"]["0"].tolist()[0]

    # Set the start and end date
    start_date = '2023-01-01'

    t = date.today()

    t = str(t)

    # Get the data
    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    test = int(time.time())
    info = finnhub_client.stock_candles(symbol, 'D', test, test)

    if info["s"] == 'no_data':
        time.sleep(3)

        print(True)

        info = finnhub_client.stock_candles(symbol, 'D', test, test)
        
        continue

    data  = pd.DataFrame(info, index= [0])
    data.columns = ["Close", "High", "Low", "Open", "S", "Timestamp", "Volume"]
    data["Datetime"] = datetime.datetime.fromtimestamp(data["Timestamp"])
    data = data.drop(columns=["S", "Timestamp"])
    data = data.set_index("Datetime")

    close = data['Close'].values[0]


    #uses api key to extract data
    params = {'apikey':api_key,
             'symbol':symbol,
             'contractType': contract,
              'strikeCount': strikecount,
              'includeQuotes': False,
              'stratgey': 'SINGLE',
              'expMonth': expMonth,
              'optionType': 'S'


             }

    content = requests.get(url = option_url, params=params)

    data = content.json()




    dates_ref = []

    int_year = now.year
    int_month = now.month
    int_day = now.day

    for i in data['putExpDateMap']:
        dates_ref.append(i)

    for i in dates_ref:

        #create outline for 
        df = {}
        date_now = i[:10]
        volume = []
        theta = []
        gamma = []
        vega = []
        delta = []
        interest = []

        #iterate through each strike
        for strike in data["putExpDateMap"][i]:
            df[strike] = {'Put': '', 'Call':''}
            df[strike]['Put'] = data["putExpDateMap"][i][strike][0]["last"]
            df[strike]['Put - Volume'] = data["putExpDateMap"][i][strike][0]["totalVolume"]

        for strike in data["callExpDateMap"][i]:

            df[strike]['Call'] = data["callExpDateMap"][i][strike][0]["last"]
            df[strike]['Call - Volume'] = data["callExpDateMap"][i][strike][0]["totalVolume"]



        label = i

        strike_date = i.split('-')

        year = int(strike_date[0])

        month = int(strike_date[1])

        day = int(strike_date[2].split(':')[0])
        
        count = int(strike_date[2].split(':')[1]) / 365

        #convert to dataframe
        option_df = pd.DataFrame.from_dict(df, orient='index')

        for row, i in option_df.iterrows():


            K = float(row)

            C = i['Call']

            P = i['Put']

            t = 'p'
            
            # obtain true put value using black scholes formula

            true_put = blackScholes(r, close, K, count, sigma, t)

            t = 'c'
            
            # obtain true call value using black scholes formula

            true_call = blackScholes(r, close, K, count, sigma, t)

            diff_put = (P - true_put) / P * 100

            diff_call = (C - true_call) / C * 100

            arb[f'{label} - {K}'] = {'Put': df[row]['Put'], 'True Put' : true_put, 'Call': df[row]['Call'], 'True Call': true_call, 'Symbol': symbol, 'Call - Volume': df[row]['Call - Volume'], 'Put - Volume': df[row]['Put - Volume'], 'Price': close, "Difference Puts": diff_put, "Difference Calls": diff_call}



KeyError: '0'

In [4]:
vol

,Unnamed: 0,Sigma,Mu
0,A,0.419616,0.000268
1,AA,0.425595,0.000083
2,AAC,0.040862,0.000045
3,AADI,1.010960,-0.001354
4,AAIC,0.722886,-0.000785
...,...,...,...
5923,YUMC,0.364256,0.000484
5924,YVR,0.984350,-0.000835
5925,YXI,0.269055,-0.000324
5926,YY,0.614916,0.000404


In [40]:
options_df = pd.DataFrame.from_dict(arb, orient='index')
options_df.head()

,Date,Strike,Put,True Put,Call,True Call,Symbol,Call - Volume,Put - Volume,Price,Difference Puts,Difference Calls,% be Below,% be above
2023-03-17:5 - 32.5,2023-03-17:5,32.5,0.29,0.105721,0.00,2.156548e+00,HAL,0,24,34.529999,63.544595,-inf,44.750343,8.702481e+01
2023-03-17:5 - 33.0,2023-03-17:5,33.0,0.43,0.186151,2.06,1.737299e+00,HAL,1,67,34.529999,56.709084,15.665092,46.098867,7.986553e+01
2023-03-17:5 - 34.0,2023-03-17:5,34.0,0.75,0.475125,1.19,1.026914e+00,HAL,46,3011,34.529999,36.650021,13.704715,48.747253,6.057814e+01
2023-03-17:5 - 35.0,2023-03-17:5,35.0,0.00,18.797569,0.00,4.707389e-110,NWS,0,0,16.180000,-inf,-inf,99.522894,5.698062e-107
2023-03-17:5 - 36.0,2023-03-17:5,36.0,1.85,1.676638,0.35,2.297085e-01,HAL,197,189,34.529999,9.370936,34.368992,53.823494,2.060734e+01


In [42]:
options_df.sort_values(['% be above', 'Call - Volume'],ascending = [False, False]).to_csv('Options.csv')

In [41]:
options_df.sort_values(['% be above', 'Call - Volume'],ascending = [False, False])

,Date,Strike,Put,True Put,Call,True Call,Symbol,Call - Volume,Put - Volume,Price,Difference Puts,Difference Calls,% be Below,% be above
2023-03-17:5 - 0.5,2023-03-17:5,0.50,0.00,0.000000e+00,2.22,2.180321e+00,LUMN,1,0,2.680000,NaN,1.787364,0.000012,1.000000e+02
2023-03-17:5 - 40.0,2023-03-17:5,40.00,0.00,3.644451e-18,0.00,1.256564e+01,BALL,0,0,52.540001,-inf,-inf,16.361361,1.000000e+02
2023-03-17:5 - 55.0,2023-03-17:5,55.00,0.05,1.634023e-26,21.00,2.578525e+01,EMN,0,0,80.750000,100.000000,-22.786900,11.137370,1.000000e+02
2023-03-17:5 - 6.35,2023-03-17:5,6.35,0.01,3.325398e-47,6.50,5.754070e+00,F,0,0,12.100000,100.000000,11.475846,4.948719,1.000000e+02
2023-03-17:5 - 7.35,2023-03-17:5,7.35,0.01,1.594549e-29,5.69,4.754711e+00,F,0,1,12.100000,100.000000,16.437418,10.115210,1.000000e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17:5 - 16.0,2023-03-17:5,16.00,0.00,5.249746e+00,0.00,5.920469e-44,AMCR,0,0,10.740000,-inf,-inf,94.470351,1.287683e-40
2023-03-17:5 - 4.5,2023-03-17:5,4.50,1.75,1.817116e+00,0.01,1.882240e-44,LUMN,0,84,2.680000,-3.835196,100.000000,94.477229,1.198065e-40
2023-03-24:12 - 6.0,2023-03-24:12,6.00,0.00,3.310775e+00,0.01,1.871763e-44,LUMN,0,0,2.680000,-inf,100.000000,99.350500,5.088606e-41
2023-03-17:5 - 5.5,2023-03-17:5,5.50,2.48,2.816475e+00,0.04,7.088948e-82,LUMN,2,0,2.680000,-13.567542,100.000000,98.656205,4.666637e-78


In [39]:
options_df.sort_values(by="Call - Volume", ascending=False)

,Date,Strike,Put,True Put,Call,True Call,Symbol,Call - Volume,Put - Volume,Price,Difference Puts,Difference Calls,% be Below,% be above
2023-03-17:5 - 152.5,2023-03-17:5,152.50,4.90,5.437132,1.06,1.534868,AAPL,25603,5229,148.500000,-10.961877,-44.798846,1.974250e-96,29.472850
2023-03-17:5 - 176.67,2023-03-17:5,176.67,7.55,6.374670,4.40,3.257898,TSLA,13812,10377,173.440002,15.567291,25.956862,4.241877e-60,37.830158
2023-05-19:68 - 41.0,2023-05-19:68,41.00,4.45,4.920418,1.02,0.796340,GM,11964,2,36.520000,-10.571196,21.927463,9.766629e-51,19.760152
2023-03-17:5 - 149.0,2023-03-17:5,149.00,2.87,3.274125,2.51,2.869618,AAPL,11462,13447,148.500000,-14.081020,-14.327408,1.974250e-96,46.387969
2023-03-31:19 - 145.0,2023-03-31:19,145.00,2.90,4.165077,6.75,8.017891,AAPL,10502,2256,148.500000,-43.623338,-18.783566,1.078389e-70,57.399463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-21:40 - 8.0,2023-04-21:40,8.00,0.10,0.000028,0.00,2.780953,AMCR,0,0,10.740000,99.972361,-inf,2.910836e-73,99.978119
2023-04-21:40 - 9.0,2023-04-21:40,9.00,0.15,0.004001,0.00,1.790042,AMCR,0,0,10.740000,97.332434,-inf,2.910836e-73,98.185514
2023-04-21:40 - 10.0,2023-04-21:40,10.00,0.10,0.082159,0.99,0.873315,AMCR,0,14,10.740000,17.841018,11.786351,2.910836e-73,79.420233
2023-04-21:40 - 13.0,2023-04-21:40,13.00,1.90,2.197690,0.05,0.004193,AMCR,0,0,10.740000,-15.667881,91.614405,2.910836e-73,0.944164


In [37]:
options_df.sort_values(by="Call - Volume", ascending=False).to_csv('Options.csv')

In [20]:
 

N_prime = norm.pdf


def vega(S, K, T, r, sigma, p):
    '''

    :param S: Asset price
    :param K: Strike price
    :param T: Time to Maturity
    :param r: risk-free rate (treasury bills)
    :param sigma: volatility
    :return: partial derivative w.r.t volatility
    '''

    ### calculating d1 from black scholes
    d1 = (np.log(S/K) + (r - p + sigma**2/2)*T)/(sigma*np.sqrt(T))

    
    vega = S  * np.sqrt(T) * N_prime(d1)
    return vega

In [6]:
vega(100, 100, 1, 0.2, 0.5)

32.29723596679143

In [17]:
def implied_volatility_call(C, S, K, T, r, tol=0.0001,
                            max_iterations=100):
    '''

    :param C: Observed call price
    :param S: Asset price
    :param K: Strike Price
    :param T: Time to Maturity
    :param r: riskfree rate
    :param tol: error tolerance in result
    :param max_iterations: max iterations to update vol
    :return: implied volatility in percent
    '''


    ### assigning initial volatility estimate for input in Newton_rap procedure
    sigma = 0.3

    for i in range(max_iterations):

        ### calculate difference between blackscholes price and market price with
        ### iteratively updated volality estimate
        diff = black_scholes_call(S, K, T, r, sigma) - C

        ###break if difference is less than specified tolerance level
        if abs(diff) < tol:
            print(f'found on {i}th iteration')
            print(f'difference is equal to {diff}')
            break

        ### use newton rapshon to update the estimate
        sigma = sigma - diff / vega(S, K, T, r, sigma)

    return sigma

In [10]:
implied_volatility_call(20.55, 100, 100, 1, 0.02, tol=0.0001,
                            max_iterations=100)

found on 2th iteration
difference is equal to -6.289563085459804e-06


0.4999687056945326

In [37]:
S = 94.65
K = 100
C = 6.275
T = 29/365
r = 0.048

In [38]:
### assigning initial volatility estimate for input in Newton_rap procedure
sigma = 0.5
diff = blackScholes(r, S, K, T, sigma, t = 'c') - C
sigma = sigma - diff / vega(S, K, T, r, sigma)
sigma

0.7770206928943932